<a href="https://colab.research.google.com/github/shubhamchourasia/LangChainBasedInvoiceReader/blob/main/Pinned_External_Financial_Advisor_Sunday_16th_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Installing packages

In [1]:
!pip install -q \
requests~=2.32.4 \
langchain~=0.3.27 \
langchain-core~=0.3.79 \
langchain-community~=0.3.31 \
langchain-google-genai~=2.0.10 \
langchain-huggingface~=0.3.1 \
langgraph~=1.0.1 \
SQLAlchemy~=2.0.44 \
chromadb==1.3.0 \
faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
  

In [2]:
!pip show \
requests \
langchain \
langchain-core \
langchain-community \
langchain-google-genai \
langchain-huggingface \
langgraph \
SQLAlchemy \
chromadb \
faker

Name: requests
Version: 2.32.5
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: bigframes, blosc2, CacheControl, community, datasets, diffusers, earthengine-api, fastai, folium, gcsfs, gdown, geocoder, google-adk, google-api-core, google-cloud-bigquery, google-cloud-storage, google-colab, google-genai, googledrivedownloader, huggingface-hub, jupyter_kernel_gateway, kaggle, kagglehub, keyrings.google-artifactregistry-auth, kubernetes, langchain, langchain-community, langsmith, libpysal, moviepy, music21, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, pandas-datareader, panel, pooch, posthog, requests-oauthlib, requests-toolbelt, spacy, spanner-graph-notebook, Sphinx, tensorflow, tensorflow-datasets, tiktoken, torchdata, transformers, tsfresh, 

## Add your Gemini's API Key in secrets

In [3]:
# Add your GEMINI_API_KEY in Secrets
from google.colab import userdata
import os

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

## Pull the helper file which already has database declarations so we don't have to dive into that

In [4]:
!wget https://raw.githubusercontent.com/NavSrijan/langchain_setup/refs/heads/master/db_setup.py

--2025-11-16 09:47:33--  https://raw.githubusercontent.com/NavSrijan/langchain_setup/refs/heads/master/db_setup.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10889 (11K) [text/plain]
Saving to: ‘db_setup.py’

db_setup.py         100%[===================>]  10.63K  --.-KB/s    in 0s      

2025-11-16 09:47:33 (106 MB/s) - ‘db_setup.py’ saved [10889/10889]



## Setting up the database

In [5]:
from db_setup import setup, get_db, User, Transaction, Goal, populate_dummy_data, TransactionType
from sqlalchemy import select, func

setup = setup()
db = get_db()
populate_dummy_data(db)


✓ Database initialized at financial_advisor.db
Populating database with 5 users, 40 transactions, and 5 goals...
✓ Dummy data populated successfully.


In [6]:
session = db.get_session()
user = session.query(User).filter(User.id == 3).first()
print(f"\nUser ID: {user.id}")
print(f"Username: {user.username}")
for x in user.transactions:
    print(f"Description: {x.description}, Amount: {x.amount}, Date: {x.transaction_date}")


User ID: 3
Username: monica85
Description: Monthly rent payment, Amount: 467.2500000000, Date: 2025-01-30 09:47:39.897017
Description: Paycheck deposit, Amount: 6650.4900000000, Date: 2025-02-04 09:47:39.920439
Description: Bar tab, Amount: 301.2800000000, Date: 2025-03-03 09:47:39.931048
Description: Vitamins, Amount: 471.9600000000, Date: 2025-06-19 09:47:39.952866
Description: Paycheck deposit, Amount: 3820.4500000000, Date: 2025-08-19 09:47:39.985737
Description: Milk, eggs, bread, Amount: 464.4800000000, Date: 2024-12-31 09:47:39.995538
Description: Streaming service, Amount: 359.9200000000, Date: 2025-10-21 09:47:40.038035
Description: Paycheck deposit, Amount: 5129.1900000000, Date: 2025-10-19 09:47:40.113152
Description: Farmer's market purchase, Amount: 94.9600000000, Date: 2025-01-11 09:47:40.170265
Description: Bi-weekly pay, Amount: 5091.9100000000, Date: 2025-03-16 09:47:40.271714


In [ ]:
# print(user.transactions)

# Basics of Langchain

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GEMINI_API_KEY)
response = llm.invoke("Explain langchain in simple terms. Give 2 line")
print(response)

content='LangChain is a framework for building applications with AI language models. It helps connect these models to external data and tools, making them more capable and useful.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--6942ca00-69cb-4405-910e-be307556efea-0' usage_metadata={'input_tokens': 11, 'output_tokens': 31, 'total_tokens': 963, 'input_token_details': {'cache_read': 0}}


## RAG
RAG combines:

    Retriever → Fetches relevant documents from a vector store.
    LLM → Generates context-aware responses using query + retrieved docs.

Today, we will:

    Store documents with Gemini Embeddings
    Retrieve docs from Chroma
    Use Gemini LLM for answering questions


## Tool calling

In [36]:
from langchain_core.tools import BaseTool
import json

retriever=vector_store.as_retriever(search_kwargs={"k":2})

class AdderTool(BaseTool):
  name: str = "adder"
  description: str = "Give two numbers 'a' and 'b', this will returns sum of a and b"

  def _run(self, tool_input: str):
    try:
      data = json.loads(tool_input)
      a,b = data.get("a"), data.get("b")
      return f"Sum of {a} and {b} is {a+b}"
    except Exception as e:
      return f"Error {e}"

class MultiplierTool(BaseTool):
  name: str ="multiplier"
  description: str = "Given two number 'a' and 'b'. Return multiplication of 'a' and 'b'"
  def _run(self, tool_input: str):
    try:
      data = json.loads(tool_input)
      a,b = data.get("a"), data.get("b")
      return f"Multiplication of {a} and {b} is {a*b}"
    except Exception as e:
      return f"Error {e}"

class RetrieverTool(BaseTool):
  name: str = "search_docs"
  description: str = "Search for relavent document"

  def _run(self, query):
    try:
      results = retriever.invoke(query)
      return results
    except Exception as e:
      return f"Error {e}"


tools = [
    AdderTool(),
    MultiplierTool(),
    RetrieverTool()
]

In [39]:
from langchain.agents import create_react_agent, AgentType, AgentExecutor, initialize_agent
from langchain_core.prompts import ChatPromptTemplate

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

res = agent.run("How much Dhoni stored in last 3 matches. Give total runs scored")
print(res)



> Entering new AgentExecutor chain...
Action: search_docs
Action Input: Dhoni runs last 3 matches
Observation: [Document(metadata={'team': 'Chennai Super Kings'}, page_content='Dhoni scored 10, 20 and 40 in the match againts Sri Lanka, Austrailia and South Africa'), Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni has led Chennai Super Kings to multiple IPL titles and is known as Captain Cool.')]
Thought:Action: adder
Action Input: {'a': 10, 'b': 20}
Observation: Error Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Thought:Action: adder
Action Input: {"a": 10, "b": 20}
Observation: Sum of 10 and 20 is 30
Thought:Action: adder
Action Input: {"a": 30, "b": 40}
Observation: Sum of 30 and 40 is 70
Thought:I now know the final answer
Final Answer: Dhoni scored a total of 70 runs in his last 3 matches.

> Finished chain.
Dhoni scored a total of 70 runs in his last 3 matches.


In [ ]:
from langchain_core.tools import BaseTool





> Entering new AgentExecutor chain...
Action: multiplier
Action Input: {"a": 128, "b": 2}{'a': 128, 'b': 2}

Observation: The product of 128 and 2 is 256
Thought:Action: divider
Action Input: {"a": 256, "b": 4}
Observation: The division of 256 and 4 is 64.0
Thought:Action: adder
Action Input: {"a": 64, "b": 10}
Observation: The sum of 64 and 10 is 74
Thought:I now know the final answer
Final Answer: 74

> Finished chain.
74


## VectorDB

In [37]:
from langchain.schema import Document

docs = [
    Document(
        page_content="Virat Kohli is one of the most successful batsmen in IPL history and has captained RCB.",
        metadata={"team": "Royal Challengers Bangalore"}
    ),
    Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, winning five titles with Mumbai Indians.",
        metadata={"team": "Mumbai Indians"}
    ),
    Document(
        page_content="MS Dhoni has led Chennai Super Kings to multiple IPL titles and is known as Captain Cool.",
        metadata={"team": "Chennai Super Kings"}
    ),
    Document(
        page_content="Jasprit Bumrah is a leading fast bowler for Mumbai Indians, famous for his yorkers.",
        metadata={"team": "Mumbai Indians"}
    ),
    Document(
        page_content="Ravindra Jadeja is an all-rounder for Chennai Super Kings, contributing with bat, ball, and fielding.",
        metadata={"team": "Chennai Super Kings"}
    ),
    Document(
        page_content="Dhoni scored 10, 20 and 40 in the match againts Sri Lanka, Austrailia and South Africa",
        metadata={"team": "Chennai Super Kings"}
    )
]


In [38]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GEMINI_API_KEY)

# Create Chroma vector store
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="rag_chroma_db",
    collection_name="ipl_docs"
)

# Add documents
vector_store.add_documents(docs)

['ed4c4019-b473-43ca-a2c2-07fb66963811',
 'bf46565c-805c-40e5-abaa-1b26a533c1e1',
 '0404fcfe-63eb-4ddc-8866-54dacbafef04',
 'e0bcca25-43ad-483b-8df1-eff9b1435cb1',
 '666f1988-5160-4df6-b4d0-29159c1e7123',
 '0ba9581d-f95e-4e7c-8300-daf3c475eceb']

In [13]:
# Initialize Retriever
retriever=vector_store.as_retriever(search_kwargs={"k":2})
#print(retriever)

res=retriever.invoke("Who is the best batsmen?")
print(res)

[Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful batsmen in IPL history and has captained RCB.'), Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is a leading fast bowler for Mumbai Indians, famous for his yorkers.')]


In [ ]:
from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     return_source_documents=True
# )

In [ ]:
# Query 1
query = "Who is the most successful IPL captain?"
response = qa_chain.invoke({"query": query})

print("Query:", query)
print("Answer:", response["result"])
print("\nSources:", [doc.metadata for doc in response["source_documents"]])



Query: Who is the most successful IPL captain?
Answer: Rohit Sharma is the most successful captain in IPL history, winning five titles with Mumbai Indians.

Sources: [{'team': 'Mumbai Indians'}, {'team': 'Royal Challengers Bangalore'}]

Query: Which bowler is famous for yorkers?
Answer: Jasprit Bumrah is famous for his yorkers.

Sources: [{'team': 'Mumbai Indians'}, {'team': 'Chennai Super Kings'}]


# Financial Advisor

We'll be creating three tools here as a demonstration:
1. List Categories
2. Sum of expense for the last X days
3. SQL Query

### List Categories

In [ ]:
# Defining a user


In [ ]:
from langchain_core.tools import BaseTool
from sqlalchemy import select, func
import json

# list categories


In [ ]:
from datetime import timedelta, datetime

# Sum of last X days


In [ ]:
from sqlalchemy import text

# SQL Tool


In [ ]:
# Initialize Agent
